In [2]:
import pandas as pd
import sys
from pickle import dump,load
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

import torch 


sys.path.append('../../')
%load_ext autoreload

In [27]:
df=pd.read_csv('../data/modified/data_to_work_on.csv')

In [5]:
df

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [105]:
%autoreload 2
from no_toxicity.src.preprocessing.cleaning import Dfcleaner

In [47]:
# Making the list of column to clean
raw_comments=df['comment_text'].to_list()

In [106]:
cleaner=Dfcleaner()


In [22]:
cleaned_comments=cleaner.clean(raw_comments,stem=False)

10000 examples cleaned out of 159571
20000 examples cleaned out of 159571
30000 examples cleaned out of 159571
40000 examples cleaned out of 159571
50000 examples cleaned out of 159571
60000 examples cleaned out of 159571
70000 examples cleaned out of 159571
80000 examples cleaned out of 159571
90000 examples cleaned out of 159571
100000 examples cleaned out of 159571
110000 examples cleaned out of 159571
120000 examples cleaned out of 159571
130000 examples cleaned out of 159571
140000 examples cleaned out of 159571
150000 examples cleaned out of 159571
Cleaning Done


In [129]:
cleaned_comments=cleaner.remove_frequent_rare(cleaned_comments,frequent=True,rare=True)

10000 examples cleaned out of 159517
20000 examples cleaned out of 159517
30000 examples cleaned out of 159517
40000 examples cleaned out of 159517
50000 examples cleaned out of 159517
60000 examples cleaned out of 159517
70000 examples cleaned out of 159517
80000 examples cleaned out of 159517
90000 examples cleaned out of 159517
100000 examples cleaned out of 159517
110000 examples cleaned out of 159517
120000 examples cleaned out of 159517
130000 examples cleaned out of 159517
140000 examples cleaned out of 159517
150000 examples cleaned out of 159517
Cleaning Done


In [131]:
# Replacing with new data
df['comment_text']=cleaned_comments

In [133]:
# # Saving the cleaned data
# df.to_csv('../data/interim/cleaned_data1.csv',index=False)

In [3]:
# Load the data
df=pd.read_csv('../data/interim/cleaned_data1.csv')
df.dropna(inplace=True)

In [4]:
def new_col_value(row):
    flag=True
    # print(row.iloc[1:])
    for i in row.iloc[1:]:
        if i==1:
            flag=False
            break
    if flag:
        return 1
    else:
        return 0


In [5]:
df['not_toxic']=df.apply(lambda row:new_col_value(row),axis=1)

In [6]:
df.iloc[:,1:].sum()

toxic             15294
severe_toxic       1595
obscene            8449
threat              478
insult             7877
identity_hate      1405
not_toxic        143240
dtype: int64

In [7]:
from imblearn.under_sampling import RandomUnderSampler

In [8]:
rus = RandomUnderSampler(random_state=42)
df,Y= rus.fit_resample(df.iloc[:,:-1],df.iloc[:,-1])
df['not_toxic']=Y

In [9]:
df

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,not_toxic
0,cocksucker piss around work,1,1,1,0,1,0,0
1,hey exclusive group wp taliban good destroy se...,1,0,0,0,0,0,0
2,bye look come think comming back tosser,1,0,0,0,0,0,0
3,gay antisemmitian archangel white tiger meow g...,1,0,1,0,1,1,0
4,fuck filthy mother as dry,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...
32445,question think troll could explain nearly edit...,0,0,0,0,0,0,1
32446,hello kyo sorry hear busy hope today find well...,0,0,0,0,0,0,1
32447,zeer oude klanken en heel nieuwe geluiden surp...,0,0,0,0,0,0,1
32448,response tord departure eddsworld follow death...,0,0,0,0,0,0,1


In [10]:
%autoreload 2
from no_toxicity.src.preprocessing.preprocess import PrepareTheData

In [11]:
dataset=PrepareTheData(df[['comment_text']],df.iloc[:,1:],max_vocab=2000)

C:\Users\dummy\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Tokenization Done
Replacing Done
token2idx created
tokens_to_indices created
Presprocessing done


In [12]:
len(dataset.token2idx_inputs)

2000

In [13]:
train_size = int(0.999 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
train_dataset.dataset.__getitem__(2000)

([1417, 204, 472, 1782], array([1, 0, 1, 0, 0, 0, 0], dtype=int64))

In [16]:

def collate(batch):
    inputs = [torch.LongTensor(item[0]) for item in batch]
    targets = [item[1] for item in batch]
    # print((inputs[0]))
    targets=torch.Tensor(targets)
    # print(targets[247])
    # Pad sequencse so that they are all the same length (within one minibatch)
    padded_inputs = pad_sequence(inputs, padding_value=dataset.token2idx_inputs[dataset.padding_token], batch_first=True)
    # print(padded_inputs.shape)
 
    # Sort by length for CUDA optimizations
    lengths = torch.LongTensor([len(x) for x in inputs])
    lengths, permutation = lengths.sort(dim=0, descending=True)
    # print(permutation)
    return padded_inputs[permutation].to(device), targets[permutation].to(device), lengths.to(device)


batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate,shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, collate_fn=collate,shuffle=True)

In [ ]:
for inputs,target,

In [33]:
%autoreload 2
from no_toxicity.src.models.encoder_decoder import LSTM

In [37]:
import torch.optim as optim
import tqdm

In [36]:
model = LSTM(input_vocab_size=len(dataset.token2idx_inputs), 
            emd_dim=100, 
            hidden_size=256,
            n_layers=1, 
            token_to_idx=dataset.token2idx_inputs)

optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=0.001)

d:\Github\no_toxicity\notebooks


In [ ]:
# Training loop
model.train()
for epoch in range(1):
    total_loss = total = 0
    progress_bar = tqdm.notebook.tqdm(train_loader, desc='Training', leave=False,total=len(train_loader))
    for inputs, targets, lengths in progress_bar:
        # Clean old gradients
        optimizer.zero_grad()

        # Forwards pass
        loss,metric = model(inputs, targets, lengths)
        # print(inputs[:,-1])
        # Perform gradient descent, backwards pass
        # break
        loss.backward()

        # Take a step in the right direction
        optimizer.step()

        # Record metrics
        total_loss += loss
        total += targets.size(1)

    train_loss = total_loss / total
    tqdm.notebook.tqdm.write(f'epoch #{epoch + 1:3d}\ttrain_loss: {train_loss:.2e}\n')